In [31]:
import pandas as pd
import json

In [32]:
dev_db = r'Dataset\Procesado\dev_db.parquet'

In [33]:
df = pd.read_parquet(dev_db)
df.sample(5)

,item_id,malo,neutral,positivo,developer,release_date,price
25745,730,0,0,1,Valve,2012-08-21,14.99
7420,550,0,1,0,Valve,2009-11-16,19.99
32268,271590,0,1,0,Rockstar North,2015-04-14,39.99
39871,319630,0,0,1,"DONTNOD Entertainment,Feral Interactive (Mac),...",2015-01-29,0.00
2543,15620,0,1,0,"Relic Entertainment,Feral Interactive (Mac/Linux)",2009-02-18,19.99


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49057 entries, 0 to 49056
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   item_id       49057 non-null  int32         
 1   malo          49057 non-null  int64         
 2   neutral       49057 non-null  int64         
 3   positivo      49057 non-null  int64         
 4   developer     49057 non-null  object        
 5   release_date  49057 non-null  datetime64[ns]
 6   price         49057 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(1)
memory usage: 2.4+ MB


## <div style="color: lightgreen; text-align: center;"> ...  def developer  ... </div>

In [ ]:
aggregation_functions = {
    'item_id': 'sum',  # Suma de la columna 'item_id' por año
    'price': lambda x: (x == 0).sum()  # Conteo de filas donde 'price' es igual a 0 por año
}

In [ ]:
resultado_por_anio =df.groupby(df['release_date'].dt.year).agg(aggregation_functions)
print(resultado_por_anio)


In [ ]:
developer = resultado_por_anio.to_json()

In [ ]:
type(resultado_por_anio)

In [ ]:
resultado_por_anio.columns = ['numero de items', 'Contenido Free']

In [ ]:
print(resultado_por_anio)

In [ ]:
#Cantidad de items
c_items = df[df['developer'] == 'Valve'].groupby(df['release_date'].dt.year)['item_id'].sum()

In [ ]:
type(c_items)

In [ ]:
df['developer'].value_counts()

## <div style="color: lightgreen; text-align: center;"> .  def best_developer_year  . </div>


In [ ]:
df.sample(5)

In [80]:
b_dev = df[df['release_date'].dt.year == 2017]

In [81]:
dev_reviews = b_dev.groupby('developer')['positivo'].sum()

In [82]:
top_devs = dev_reviews.nlargest(3)


In [83]:
top_dev_json = top_devs.to_json()

In [84]:
print(top_dev_json)

{"Smartly Dressed Games":632,"Freejam":217,"Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC":188}


## <div style="color: lightgreen; text-align: center;"> .  def developer_reviews_analysis  . </div>

In [90]:
dev_d = df[df['developer'] == 'Valve']

In [91]:
dev_r = dev_d.groupby('developer')[['positivo', 'malo']].sum()

In [92]:
dev_j = dev_r.to_json()

In [ ]:
result = 

In [93]:
print(dev_j)

{"positivo":{"Valve":6056},"malo":{"Valve":1098}}


In [95]:
dev = 'Valve' # Variable event de lambda
d_data = df[df['developer'] == dev]

neg_r = d_data['malo'].sum()
pos_r = d_data['positivo'].sum()

result = {dev:{'Negativo': neg_r, 'Positivo': pos_r}}


result_df = pd.DataFrame(result).T  # Convertir el diccionario en un DataFrame transpuesto
result_json = result_df.to_json()

print(result)

{'Valve': {'Negativo': 1098, 'Positivo': 6056}}


## <div style="color: lightgreen; text-align: center;"> .  def user_data  . </div>

In [147]:
user_db = r'Dataset\Procesado\user_db.parquet'

In [148]:
df = pd.read_parquet(user_db)
df.head(5)

,price,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Sports,Strategy,Utilities,Video Production,Web Publishing,item_id,user_id,review_sum,item_name,playtime_forever
0,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-GM-Dragon,2.0,Counter-Strike,14.0
1,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-Mad-,2.0,Counter-Strike,8.0
2,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-SEVEN-,9.0,Counter-Strike,617.0
3,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-SatansLittleHelper-,1.0,Counter-Strike,511.0
4,9.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,-Ultrix,1.0,Counter-Strike,5.0


In [149]:
user = '-Mad-'

In [150]:
dfm = df[df['user_id'] == user]

In [151]:
money = dfm.groupby('user_id')['price'].sum()
items = dfm.groupby('user_id')['item_id'].value_counts().sum()

In [159]:
result = {"Usuario X": user, "Dinero gastado": float(money.values[0]), "Cantidad de items": int(items)}

In [160]:
print(result)

{'Usuario X': '-Mad-', 'Dinero gastado': 3638.23, 'Cantidad de items': 296}


In [ ]:
#Lambda Function

# user = '-Mad-'

# dfm = df[df['user_id'] == user]

# money = dfm.groupby('user_id')['price'].sum()
# items = dfm.groupby('user_id')['item_id'].value_counts().sum()

# result = {"Usuario X": user, "Dinero gastado": float(money.values), "Cantidad de items": int(items)}

# print(result)

## <div style="color: lightgreen; text-align: center;"> .  def UserForGenre  . </div>

In [130]:
df.sample(5)

,price,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Sports,Strategy,Utilities,Video Production,Web Publishing,item_id,user_id,review_sum,item_name,playtime_forever
455071,24.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,219640,plumbers,4.0,Chivalry: Medieval Warfare,56.0
144442,14.99,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16450,Lad123,2.0,F.E.A.R. 2: Project Origin,0.0
1876309,19.99,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,620,76561198055819674,5.0,Portal 2,184.0
1819482,19.99,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,50620,Thisbananaistastyfam,6.0,Darksiders,0.0
482013,9.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,220780,76561198079765149,1.0,Thomas Was Alone,177.0


In [128]:
genre_x = "Action"

In [129]:
dfg = df[genre_x]

In [131]:
horas_j = dfm.groupby('user_id')['playtime_forever'].sum()

In [133]:
user_horas_j = horas_j.idxmax()

In [165]:
resp = {f"Usuario con más horas jugadas para {genre_x}": user_horas_j, "Total horas jugadas": float(horas_j.values[0])}

In [166]:
print(resp)


{'Usuario con más horas jugadas para Action': '-Mad-', 'Total horas jugadas': 175458.0}


In [ ]:
genre_x = "Action"

dfg = df[genre_x]

horas_j = dfm.groupby('user_id')['playtime_forever'].sum()
user_horas_j = horas_j.idxmax()

resp = {f"Usuario con más horas jugadas para {genre_x}": user_horas_j, "Total horas jugadas": float(horas_j.values[0])}